In [1]:
#pip install import_ipynb

In [1]:
import os
import numpy as np
#from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize


In [3]:
def generate_id_mapping(characters):
    char_to_id = {char: idx for idx, char in enumerate(characters)}
    id_to_char = {idx: char for idx, char in enumerate(characters)}
    return char_to_id, id_to_char

def generate_char_one_hot_vector(char_id, char_vector_size=45):
    char_vector = np.zeros(char_vector_size)
    char_vector[char_id] = 1
    return char_vector

def one_hot_encode_chars(char, char_to_id, char_vector_size=45):
    char_id = char_to_id.get(char, None)

    if char_id is not None:
        return generate_char_one_hot_vector(char_id, char_vector_size)
    else:
        return None

In [4]:
import numpy as np

def get_word_embeddings(words, word2vec_model):
    word_embeddings = []
    for word in words:
        # Embeddings for individual words in the list
        embeddings = []
        for subword in word:
            if subword in word2vec_model.wv:
                embeddings.append(word2vec_model.wv[subword])
            else:
                # Handle out-of-vocabulary words
                embeddings.append(np.zeros(word2vec_model.vector_size))
        word_embeddings.append(embeddings)
    return word_embeddings


In [5]:
allowed_characters = [
    'ء', 'إ', 'أ', 'ا', 'آ','ﺁ', 'ب', 'پ', 'ت', 'ث', 'ج', 'چ', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط',
    'ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'ھ',
    'ؤ', 'و', 'ي', 'ئ', 'يء','ى', 'ة', 'ﻵ', 'ﻻ', 'ﻹ', 'ﻷ',
    'َ', 'ُ', 'ِ', 'ّ', 'ْ', 'ٌ', 'ً', 'ٍ'
]

In [6]:
def clean_text(text):
    
    # Remove characters not in the allowed list
    cleaned_text = ''.join(char if char in allowed_characters else ' ' for char in text)
    
    # Collapse consecutive spaces into a single space
    cleaned_text = ' '.join(cleaned_text.split())

    return cleaned_text

In [7]:
def tokenize_text(text):
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    return tokens

In [8]:
# Extract letters in X and corresponding Diacritics in Y and same for words
def extract_arabic_and_Diacritics(arabic_tokens):
    # List of characters without Diacritics
    characters = []
    # List of words without Diacritics
    words = []
    # List of Diacritics corresponding to each character in characters list
    Diacritics_characters = []
    # List of Diacritics corresponding to each word in words list
    Diacritics_words = []

    # All Diacritics letters in Arabic
    Diacritics_set = ('َ', 'ُ', 'ِ', 'ّ', 'ْ', 'ٌ', 'ً', 'ٍ')

    # For each word with Diacritics
    for token in arabic_tokens:
        # Will be concatinated with letters without Diacritics
        word_without_Diacritics = ''
        # Check if the last letter in the word is Diacritics ==> Add it to Diacritics_words
        if token[-1] in Diacritics_set:
            Diacritics_words.append(token[-1])
        # If the last character isn't Diacritics ==> Add None to Diacritics_words
        else:
            Diacritics_words.append(None)

        # Loop over all characters of this word
        for idx, char in enumerate(token):
            # Check it isn't Diacritics (It is Arabic characher)
            if char not in Diacritics_set:
                # Add the character without Diacritics to characters list
                characters.append(char)
                # Concatinate the character without Diacritics to the word
                word_without_Diacritics += char

                # Extract Diacritics (it's the character after the Arabic letter)
                Diacritics_index = idx + 1
                # A list because it can be more than one Diacritics characters
                Diacritics = []
                # Check it is a Diacritics character
                if Diacritics_index < len(token) and token[Diacritics_index] in Diacritics_set:
                    Diacritics.append(token[Diacritics_index])
                    Diacritics_index +=1
                    # Diacritics may be 2 characters if it is (شدة + فتحة/ضمة/كسرة)
                    # Check for the character after the Diacritics character
                    if Diacritics_index < len(token) and token[Diacritics_index] in Diacritics_set:
                        Diacritics.append(token[Diacritics_index])
                    else:
                        # The second Diacritics character is None
                        Diacritics.append(None)
                else:
                    # No Diacritics
                    Diacritics.extend([None, None])
                # Add the Diacritics of the character to the list item corresponding to the character in characters list
                Diacritics_characters.append(Diacritics)
        # Add the word whithout Diacritics to the words list
        words.append(word_without_Diacritics)
        
                
    return characters, Diacritics_characters, words, Diacritics_words

In [9]:
# Specify the path to your dataset
dataset_path = "dataset/train.txt"

# Read and preprocess the data
with open(dataset_path, 'r', encoding='utf-8') as file:
    lines = [line.strip() for line in file.readlines()]

# Split each line on dots to minimize the length of the line
split_lines = [line.split('.') for line in lines]
# Flatten the lines after splitting so each splitted part is a separate line
# Remove empty lines
flat_split_lines = [part for line in split_lines for part in line if part.strip()]

# Lists to contain all the training data and its labels
X_letters_list = []
Y_letters_list = []
X_words_list = []
Y_words_list = []

tokenized_lines = []

# Test the functions on the first 10 lines
for line in flat_split_lines:
    cleaned_line = clean_text(line)
    removed = set(line) - set(cleaned_line)
    tokenized_line = tokenize_text(cleaned_line)

    X_letters, Y_letters, X_words, Y_words = extract_arabic_and_Diacritics(tokenized_line)
    X_letters_list.append(X_letters)
    Y_letters_list.append(Y_letters)
    X_words_list.append(X_words)
    Y_words_list.append(Y_words)
    tokenized_lines.append(tokenized_line)
    
    


In [10]:
print(X_words_list[0])
print(Y_letters_list[0])
print(tokenized_lines[0])

['قوله', 'أو', 'قطع', 'الأول', 'يده', 'إلخ', 'قال', 'الزركشي']
[['َ', None], ['ْ', None], ['ُ', None], ['ُ', None], ['َ', None], ['ْ', None], ['َ', None], ['َ', None], ['َ', None], [None, None], ['ْ', None], ['َ', None], ['ّ', 'َ'], ['ُ', None], ['َ', None], ['َ', None], ['ُ', None], [None, None], ['َ', None], ['ْ', None], ['َ', None], [None, None], ['َ', None], [None, None], [None, None], ['ّ', 'َ'], ['ْ', None], ['َ', None], ['ِ', None], ['ّ', 'ُ']]
['قَوْلُهُ', 'أَوْ', 'قَطَعَ', 'الْأَوَّلُ', 'يَدَهُ', 'إلَخْ', 'قَالَ', 'الزَّرْكَشِيُّ']


In [11]:
# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X_words_list, vector_size=100, window=5, min_count=1, workers=4)

# Extract features using Word2Vec
word_embeddings = get_word_embeddings(X_words_list, word2vec_model)

In [12]:
characters = set([char for sequence in X_letters_list for char in sequence])

char_to_id, _ = generate_id_mapping(list(characters))

# Testing the function on the X_letters_list
X_letters_one_hot = { char: one_hot_encode_chars(char, char_to_id, len(characters)) for char in characters }

# Display the results for the first few characters
# for char, one_hot_encoding in X_letters_one_hot.items():
#     print(f"Character: {char}, One-Hot Encoding: {one_hot_encoding} - Length: {len(one_hot_encoding)}")

In [13]:
import pickle

with open('diacritics.pickle', 'rb') as file:
    diacritics = pickle.load(file)
    
with open('diacritic2id.pickle', 'rb') as file:
    diacritic2id = pickle.load(file)
    
with open('arabic_letters.pickle', 'rb') as file:
    arabic_letters = pickle.load(file)
    
print(diacritics)
print(diacritic2id)
print(arabic_letters)

print(len(diacritics))
print(len(diacritic2id))
print(len(arabic_letters))



{'ِ', 'ٌ', 'ً', 'ٍ', 'ُ', 'ْ', 'ّ', 'َ'}
{'َ': 0, 'ً': 1, 'ُ': 2, 'ٌ': 3, 'ِ': 4, 'ٍ': 5, 'ْ': 6, 'ّ': 7, 'َّ': 8, 'ًّ': 9, 'ُّ': 10, 'ٌّ': 11, 'ِّ': 12, 'ٍّ': 13, '': 14}
{'ط', 'د', 'ك', 'ظ', 'ش', 'أ', 'خ', 'ض', 'ذ', 'ة', 'ص', 'ت', 'م', 'ه', 'ح', 'ج', 'ع', 'ن', 'آ', 'ر', 'غ', 'ئ', 'ز', 'ي', 'إ', 'ء', 'ى', 'ؤ', 'و', 'ق', 'ب', 'ث', 'س', 'ف', 'ل', 'ا'}
8
15
36


#### Build Feature Vectors with Hot Vector

In [14]:
joined_sentences = ["".join(sentence) for sentence in X_words_list]

X = []
Y = []

for sentence_list, diacritics_list in zip(joined_sentences, Y_letters_list):
    length = len(sentence_list)
    for idx in range(length):
        X.append(X_letters_one_hot[sentence_list[idx]])
        d = ''
        if diacritics_list[idx][0] != None:
            d += diacritics_list[idx][0]
            if diacritics_list[idx][1] != None:
                d += diacritics_list[idx][1]
            
        Y.append(diacritic2id[d])
        
with open('X.pickle', 'wb') as file:
    pickle.dump(X, file)
    
with open('Y.pickle', 'wb') as file:
    pickle.dump(Y, file)
    
print(X[3])
print(Y[3])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
2


#### Build Feature Vectors with Hot Vector + Word2Vec Embedding

In [ ]:
X = []
Y = []

for i, sentence in enumerate(X_words_list):
    char_index = 0
    diacritics_list = Y_letters_list[i]
    for j, word in enumerate(sentence):
        word_vector = word_embeddings[i][j]
        for k, char in enumerate(word):
            char_hot_vector = X_letters_one_hot[char]
            X.append(list(char_hot_vector) + list(word_vector))
            d = ''
            if diacritics_list[char_index][0] != None:
                d += diacritics_list[char_index][0]
                if diacritics_list[char_index][1] != None:
                    d += diacritics_list[char_index][1]
            
            Y.append(diacritic2id[d])
            char_index += 1
            
print(X[6])
print(Y[6])

57778


In [69]:
with open('X_word2vec.pickle', 'rb') as file:
    x = pickle.load(file)
    
# with open('Y.pickle', 'rb') as file:
#     y = pickle.load(file)
    
# print(x[:10])
# print(y[:10])

[array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]

## Bag of Words

In [17]:
word_diacritic_freq = {}

for i, words in enumerate(X_words_list):
    for j, word in enumerate(words):
        diacritic = Y_words_list[i][j]
        prev_freq = word_diacritic_freq.get((word, diacritic), 0)
        word_diacritic_freq.__setitem__((word, diacritic), prev_freq + 1)

In [18]:
print(list(word_diacritic_freq.items())[:50])

[(('قوله', 'ُ'), 20797), (('أو', 'ْ'), 31767), (('قطع', 'َ'), 598), (('الأول', 'ُ'), 685), (('يده', 'ُ'), 332), (('إلخ', 'ْ'), 4808), (('قال', 'َ'), 16517), (('الزركشي', 'ُ'), 228), (('ابن', 'ُ'), 3789), (('عرفة', 'َ'), 495), (('بلفظ', 'ٍ'), 41), (('يقتضيه', None), 11), (('كإنكار', 'ِ'), 1), (('غير', 'ِ'), 3009), (('حديث', 'ٍ'), 38), (('بالإسلام', 'ِ'), 56), (('وجوب', 'َ'), 105), (('ما', None), 17401), (('علم', 'َ'), 1067), (('وجوبه', 'ُ'), 64), (('من', 'ْ'), 44648), (('الدين', 'ِ'), 973), (('ضرورة', 'ً'), 44), (('كإلقاء', 'ِ'), 4), (('مصحف', 'ٍ'), 25), (('بقذر', 'ٍ'), 2), (('وشد', 'ِ'), 24), (('زنار', 'ٍ'), 2), (('قول', 'ُ'), 1393), (('ابن', 'ِ'), 2338), (('شاس', 'ٍ'), 91), (('بفعل', 'ٍ'), 43), (('يتضمنه', 'ُ'), 5), (('هو', 'َ'), 5218), (('كلبس', 'ِ'), 8), (('الزنار', 'ِ'), 1), (('وإلقاء', 'ِ'), 2), (('المصحف', 'ِ'), 49), (('في', None), 46154), (('صريح', 'ِ'), 6), (('النجاسة', 'ِ'), 74), (('والسجود', 'ِ'), 12), (('للصنم', 'ِ'), 1), (('ونحو', 'ِ'), 105), (('ذلك', 'َ'), 12839), (('وسحر'

In [21]:
char_diacritic_freq = {}

for i, chars in enumerate(X_letters_list):
    for j, char in enumerate(chars):
        diacritic = tuple(Y_letters_list[i][j])
        prev_freq = char_diacritic_freq.get(char, {})
        freq = prev_freq.get(diacritic, 0)
        prev_freq.__setitem__(diacritic, freq + 1)
        char_diacritic_freq.__setitem__(char, prev_freq)

In [24]:
print(list(char_diacritic_freq.items())[0])

('ق', {('َ', None): 148128, ('ْ', None): 26576, ('ِ', None): 32778, ('ُ', None): 23409, ('ّ', 'َ'): 3368, ('ّ', 'ِ'): 2848, ('ّ', 'ٍ'): 371, ('ٌ', None): 2184, ('ّ', 'ُ'): 2235, ('ٍ', None): 1860, ('ً', None): 2315, ('ّ', 'ً'): 232, ('ّ', None): 50, ('ّ', 'ٌ'): 378, (None, None): 256})


In [29]:
diacritics_probability = {}

total = 0

for char, freq in char_diacritic_freq.items():
    for diacritic, f in freq.items():
        updated_freq = diacritics_probability.get(diacritic, 0) + f
        total += f
        diacritics_probability.__setitem__(diacritic, updated_freq)
        
for diacritic, f in diacritics_probability.items():
    diacritics_probability[diacritic] /= total
    
print(diacritics_probability)

{('َ', None): 0.3511366491057032, ('ْ', None): 0.1471525159977671, ('ِ', None): 0.15130519412252538, ('ُ', None): 0.09701240906100693, ('ّ', 'َ'): 0.035844792981553686, ('ّ', 'ِ'): 0.0076095512674523, ('ّ', 'ٍ'): 0.0006978405499002691, ('ٌ', None): 0.006951583899281061, ('ّ', 'ُ'): 0.005184591278334206, ('ٍ', None): 0.010460783109289159, ('ً', None): 0.006993013691708222, ('ّ', 'ً'): 0.00041381896713372173, ('ّ', None): 0.0007520824457658871, ('ّ', 'ٌ'): 0.0005680431655330949, (None, None): 0.1779171303570458}


In [52]:
def classify_character(char):
    max_prob = 0
    best_diacritic = None
    for diacritic in diacritics_probability.keys():
        count = char_diacritic_freq[char].get(diacritic, 0)
        total_count = diacritics_probability[diacritic] * total
        prob = diacritics_probability[diacritic] * count / total_count
        if prob > max_prob:
            max_prob = prob
            best_diacritic = diacritic
    return best_diacritic


In [72]:
text = 'يده'
output = ''

for idx, char in enumerate(text):
    if char == ' ':
        output += ' '
        continue
    best_diacritic = classify_character(char)
    output += char
    if best_diacritic[0]:
        output += best_diacritic[0]
        if best_diacritic[1]:
            output += best_diacritic[1]
            
print(output)


يدَهُ
